## EDA и предобработка данных

Удобно представить данные не только в виде графа, но и в виде квадратной sparse-матрицы взаимодействия пользователей. 

```python 
    def _create_interaction_matrix(self, df, target_col='target') -> pd.DataFrame:
        """
        Created square iteration matrix (NaN values updated using transposed values)
        Parameters
        ----------
        df : pd.DataFrame
            Users interaction history dataset
        target_col : str
            Label of target column (interaction intensity, time)
        Returns
        -------
        interaction_matrix : pd.Dataframe
            Matrix of full users interactions
        """
        interaction_matrix = df[['uid1', 'uid2', target_col]].pivot(index='uid1', columns='uid2')
        interaction_matrix.columns = interaction_matrix.columns.droplevel(0)
        full_index = interaction_matrix.index.union(interaction_matrix.columns).sort_values()
        interaction_matrix = interaction_matrix.reindex(index=full_index, columns=full_index)
        interaction_matrix.update(interaction_matrix.T)
        if self.idf_normalize:
            vc = df[['uid1', 'uid2']].stack().value_counts(normalize=True)
            idf = np.log(1 / vc).sort_index()
            interaction_matrix = interaction_matrix.multiply(idf, axis=1)
        return interaction_matrix.fillna(0)
```

## Recommender system model

В качестве первой модели для рекомендательной системы я использовал модифицированный kNN с метрикой `cosine_distance`. Для каждой пары пользователей мы может вычислить скалярное произведение, соответственно вычислив матрицу скалярный произведений, отсеяв уже добавленных друзей и отранжировав её можно получить упорядоченный список похожих (в терминах близости векторов) пользователей. Для пользователей, впервые появившихся на тесте, такой метод применить не получится (по крайней мере, сложно на таком объёме данных обобщить пользователей другим способом), сответственно для `cold_users` я использовал вершины с наибольшим количеством связей из графа G.

```python 
    def fit(self):
        self.model = linear_kernel(self.train_csr, self.train_csr)

    def predict(self, users_list, k=5) -> np.array:
        """
        Computes first k recomendations for users in list
        Parameters
        ----------
        users_list : np.array
            An array of users uids who needs recommendations
        k : int, optional
            The maximum number of predicted elements
        Returns
        -------
        recommendations : np.array
            Ordered array with recommended users uids
        """
        if self.ind2uid.isin(users_list).any():
            valid_idx = self.uid2ind[users_list]
            cold_start = np.argwhere(np.isnan(valid_idx)).flatten()
            valid_idx = valid_idx.dropna().astype(int).values
            model_recs = self.rec_calc(valid_idx)
            users_list = self.ind2uid[valid_idx]
            for i, (ind, uid) in tqdm(enumerate(zip(valid_idx, users_list))):
                model_recs[i, ind] = -1
                if uid in self.neighbors:
                    friends_uids = self.neighbors[uid]
                    friends_idx = self.uid2ind.loc[friends_uids].dropna().astype(int).values
                    model_recs[i, friends_idx] = -1
            sorted_recs = model_recs.argsort()[:, ::-1]
            final_recs = list(map(lambda x: self.ind2uid.loc[x].values, sorted_recs[:, :k]))
            for cold_idx in cold_start:
                baseline_recommendation = self.find_popular[:k]
                final_recs = np.insert(final_recs, cold_idx, baseline_recommendation, axis=0)
            return final_recs
        else:
            baseline_recommendation = self.find_popular[:k]
            return np.array([baseline_recommendation] * len(users_list))
        
    def rec_calc(self, idx) -> np.array:
        recommendations = self.model[idx]
        return recommendations
```


В качестве второй модели для рекомендательной системы я использовал SVD разложение матрицы взаимодействий. Теоретически, после обратного преобразования мы должны получить хорошую аппроксимацию исходной матрицы, но я столкнулся с проблемой (`popularity_bias`?), связанной с тем, что пользователи с наибольшим числом связей перевешивали остальных. Пытался регуляризировать с помощью домножения на `TF-IDF`, добавить коэффициент регуляризации и откалибровать его на тестовой выборке, подобрать нужную метрику. 

```python 
    def fit(self):
        self.model = TruncatedSVD(random_state=self.random_state)
        self.model.fit(self.train_csr)
        self.uid1_repres = self.model.transform(self.train_csr)
        self.uid2_repres = self.model.components_

    def rec_calc(self, idx) -> np.array:
        recommendations = np.dot(self.uid1_repres[idx, :], self.uid2_repres[:, :])
        return recommendations
```

# Проблемы / вопросы / дальнейшие улучшения

Основной проблемой остался подбор нужной метрики для восстановления матрицы после SVD. Также постфактум выяснилось, что готовая модель после сереализации весит около 15Гб, дешевле запускать модель с нуля на уже предобработанных датасетах: загрузка сереализованного объекта длится около минуты, построение модели около 20 секунд. Для большого количества предсказаний работает достаточно эффективно, для совсем небольшого задержка для запуска модели уже становится значимой. Основной целью было реализовать систему для эффективной валидации (30% датасета) - на лучшей модели она отрабатывает за 30 секунд, на лучшей эвристике - за минуту.  

В качестве следующей модели начал реализовывать обучение с учителем, планирую доделать на выходных - самому интересно, что получится. Пайплайн в принципе понятен: для пар юзеров нагенерить фичей на графах (JaccardCoefficent, ResourceAllocation, AdamicAdar, PreferentialAttachment, CommonNeighbors), после чего добавить аугментаций (например, пары друзей предсказанные kNN и тд) и уже на этом обучить бустинг, предсказания также ранжировать по predict_proba, либо по значению предсказанного intensity. 